# NYC Weather Data
The NYC weather data is from the National Centers for Environmental Information, https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094728/detail. The data is from the weather station at `NY CITY CENTRAL PARK, NY US` from `1869-01-01` to `2023-02-27`. And from the dates we need weather data, from 2006-01-01 to 2021-12-31, we see that are no null values. So there's minimal processing besides converting the date string to datetime, filtering by a time range, and selecting the columns we want.

## Data Dictionary
The data dictionary is available in the link above under the `Available Data Types` in `Station Data Inventory, Access & History`. It will show that `TAVG`, `TMIN`, `TMAX` is average, minimum, and maximum temperature respectively.

## Purpose
The only columns I'll use for in weather data is average temperature, minimum temperature, maximum temperature, precipitation, and snow fall. I think these columns will be meaningful to see the correlation between crime rates in NYC and the variables.

In [23]:
from os.path import join

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots

from pyspark.sql.dataframe import DataFrame as PySparkDataFrame

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    count, 
    when, 
    lit,
    isnan,
    col,
    concat_ws,
    from_unixtime,
    unix_timestamp,
    to_timestamp,
    year, 
    month,
    dayofmonth,
    hour
)

import statsmodels.formula.api as smf

pio.renderers.default = "iframe"

In [2]:
def read_weather_data(fname: str=join("..", "data", "USW00094728.csv")) -> pd.DataFrame:
    df = pd.read_csv(fname, parse_dates=["DATE"])
    subset_df = df[["DATE", "TAVG", "TMAX", "TMIN", "PRCP", "SNOW"]]
    timerange = subset_df.query("DATE.between('2006-01-01', '2021-12-31')")
    return timerange.reset_index(drop=True)

In [3]:
df = read_weather_data()
df

,DATE,TAVG,TMAX,TMIN,PRCP,SNOW
0,2006-01-01,4.93,8.64,1.23,126.8,51.0
1,2006-02-01,2.07,5.78,-1.63,73.1,683.0
2,2006-03-01,6.15,10.33,1.98,20.3,33.0
3,2006-04-01,13.15,18.48,7.82,141.2,3.0
4,2006-05-01,17.27,21.97,12.57,117.5,0.0
...,...,...,...,...,...,...
187,2021-08-01,25.30,28.81,21.80,262.1,0.0
188,2021-09-01,21.27,24.79,17.76,254.8,0.0
189,2021-10-01,16.67,19.93,13.41,133.7,0.0
190,2021-11-01,7.91,11.54,4.29,28.6,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    192 non-null    datetime64[ns]
 1   TAVG    192 non-null    float64       
 2   TMAX    192 non-null    float64       
 3   TMIN    192 non-null    float64       
 4   PRCP    192 non-null    float64       
 5   SNOW    192 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 9.1 KB


In [5]:
df.describe()

,TAVG,TMAX,TMIN,PRCP,SNOW
count,192.000000,192.000000,192.000000,192.000000,192.000000
mean,13.373802,17.307656,9.439583,111.375521,66.770833
std,8.664113,8.979786,8.384961,61.889651,161.756101
min,-4.410000,0.080000,-8.900000,9.200000,0.000000
25%,5.407500,9.287500,1.572500,71.000000,0.000000
50%,13.310000,17.595000,9.225000,102.750000,0.000000
75%,21.670000,25.862500,17.615000,138.675000,34.250000
max,27.410000,32.270000,22.970000,481.300000,937.000000


In [6]:
df.isna().any()

DATE    False
TAVG    False
TMAX    False
TMIN    False
PRCP    False
SNOW    False
dtype: bool

## Crime Data

In [7]:
def initializeSpark() -> SparkSession:
    """Create a Spark Session for Streamlit app"""
    conf = SparkConf().setAppName("crime-processor").setMaster("local")
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    return spark, spark.sparkContext

In [8]:
def getCrimesPerMonth(sdf: PySparkDataFrame) -> pd.DataFrame:
    crimeTimes = sdf\
    .select(
        year("CMPLNT_FR").alias("CMPLNT_FR_YEAR"),
        month("CMPLNT_FR").alias("CMPLNT_FR_MONTH"),
        dayofmonth("CMPLNT_FR").alias("CMPLNT_FR_DAY"),
        hour("CMPLNT_FR").alias("CMPLNT_FR_HOUR")
    ).cache()
    
    crimesPerMonth = crimeTimes\
        .groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
        .count()\
        .sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
        .toPandas()
    
    crimesPerMonth["Date"] = crimesPerMonth.CMPLNT_FR_MONTH.map(str) + "/" + crimesPerMonth.CMPLNT_FR_YEAR.map(str)
    crimesPerMonth.Date = pd.to_datetime(crimesPerMonth.Date)
    crimesPerMonth.set_index("Date", inplace=True)
    crimesPerMonth.drop(columns=["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"], inplace=True)
    return crimesPerMonth

In [9]:
spark, _ = initializeSpark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 15:56:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/07 15:56:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
processedSDF = spark.read.load(path=join("..", "data", "NYPD_Complaint_Data_Historic.parquet"), format="parquet")
processedSDF.limit(5).toPandas()

,CMPLNT_NUM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,...,SUSP_RACE,SUSP_SEX,LATITUDE,LONGITUDE,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CMPLNT_TO,CMPLNT_FR
0,506547392,32,2018-03-30 00:00:00,351,CRIMINAL MISCHIEF,254,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,MANHATTAN,...,None,None,40.810877,-73.941064,PATROL BORO MAN NORTH,25-44,WHITE,F,None,2018-03-29 20:30:00
1,629632833,52,2018-02-07 00:00:00,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,BRONX,...,BLACK,F,40.873671,-73.908014,PATROL BORO BRONX,UNKNOWN,UNKNOWN,D,None,2018-02-06 23:15:00
2,787203902,75,2018-11-21 00:00:00,341,PETIT LARCENY,321,"LARCENY,PETIT FROM AUTO",COMPLETED,MISDEMEANOR,BROOKLYN,...,WHITE HISPANIC,F,40.651782,-73.885457,PATROL BORO BKLYN NORTH,UNKNOWN,UNKNOWN,D,2018-11-21 00:20:00,2018-11-21 00:15:00
3,280364018,10,2018-06-10 00:00:00,361,OFFENSES AGAINST PUBLIC ORDER/ADMINISTRATION,639,AGGRAVATED HARASSMENT 2,COMPLETED,MISDEMEANOR,MANHATTAN,...,WHITE HISPANIC,M,40.759310,-73.994706,PATROL BORO MAN SOUTH,18-24,WHITE HISPANIC,F,2018-06-09 21:43:00,2018-06-09 21:42:00
4,985800320,19,2018-11-10 00:00:00,341,PETIT LARCENY,333,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,MANHATTAN,...,BLACK HISPANIC,F,40.764536,-73.970728,PATROL BORO MAN NORTH,UNKNOWN,UNKNOWN,D,2018-11-10 19:45:00,2018-11-10 19:40:00


In [11]:
crimesPerMonth = getCrimesPerMonth(sdf=processedSDF)

## Compare Crime Rate and Temperature

In [12]:
def compareCrimeRateAndTemperature(weatherData: pd.DataFrame, crimeData: pd.DataFrame):
    fig = make_subplots(
        rows=2, 
        cols=1,
        subplot_titles=("Temperature", "Crime Rate"),
        shared_xaxes=True
    )

    average_temperature = go.Scatter(
        x=weatherData.DATE,
        y=weatherData.TAVG,
        hovertemplate="<i>Date</i>: %{x}"
              "<br><i>Temperature</i>: %{y}°C<br>"
              "<extra></extra>",
        mode="lines",
        name="Average Temperature"
    )


    monthlyCrime = go.Scatter(
        x=crimeData.index,
        y=crimeData["count"],
        hovertemplate="<i>Date</i>: %{x}"
          "<br><i>Count</i>: %{y}<br>"
          "<extra></extra>",
        mode="lines",
        name="Monthly Crime Rates"
    )

    fig.add_trace(average_temperature, row=1, col=1)
    fig.add_trace(monthlyCrime, row=2, col=1)


    # Update xaxis properties
    fig.update_xaxes(title_text="Month", row=2, col=1)

    # Update yaxis properties
    fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)

    fig.update_layout(
        title="Temperature and Crime Rates from 2006-2021",
    )
    return fig

In [13]:
compareCrimeRateAndTemperature(weatherData=df, crimeData=crimesPerMonth)

Here we see seasionality that it's hotter in the summer months (June, July, August) and colder in the winter months (December, January, February)

## How Does Temperature Affect Crime Rates?
* How strong is the correlation? Inside vs Outside?
* Which type of crimes are most affected by temeprature?

We see that temperature is a poor indicator of crime rates. The r-squared value, which ranges from 0 to 1, is 0.28. This indicates that the linear regression model poorly fits the data. Perhaps we should change the scope from finding the correlation of temperature and crime rates to finding correlation of temperature and crimes that have occurred outside. 

In [14]:
fig = px.scatter(
    x=df.TAVG, 
    y=crimesPerMonth["count"],
    trendline="ols",
    title="Crimes per Month vs. Temperature",
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Crime Rate",
)
fig

In [22]:
processedSDF.select("LOC_OF_OCCUR_DESC").distinct().show()

+-----------------+
|LOC_OF_OCCUR_DESC|
+-----------------+
|      OPPOSITE OF|
|          REAR OF|
|             null|
|           INSIDE|
|         FRONT OF|
+-----------------+



In [32]:
outsideCrimes = processedSDF\
    .select(
        "LOC_OF_OCCUR_DESC",
        year("CMPLNT_FR").alias("CMPLNT_FR_YEAR"),
        month("CMPLNT_FR").alias("CMPLNT_FR_MONTH"),
    )\
    .filter((col("LOC_OF_OCCUR_DESC") != "INSIDE")).cache()

insideCrimes = processedSDF\
    .select(
        "LOC_OF_OCCUR_DESC",
        year("CMPLNT_FR").alias("CMPLNT_FR_YEAR"),
        month("CMPLNT_FR").alias("CMPLNT_FR_MONTH"),
    )\
    .filter((col("LOC_OF_OCCUR_DESC") == "INSIDE"))

In [16]:
outsideCrimesDF = outsideCrimes.groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
    .count()\
    .sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
    .toPandas()

insideCrimesDF = insideCrimes.groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
    .count()\
    .sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
    .toPandas()

In [20]:
fig = px.scatter(
    x=df.TAVG, 
    y=insideCrimesDF["count"],
    trendline="ols",
    title="Indoor Crimes per Month vs. Temperature",
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Crime Rate",
)
fig

In [21]:
fig = px.scatter(
    x=df.TAVG, 
    y=outsideCrimesDF["count"],
    trendline="ols",
    title="Outdoor Crimes per Month vs. Temperature",
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Crime Rate",
)
fig

Are there any crimes that correlate stronger to temperature than other type of crimes? Let's see if we can sort crimes by how strong they correlate with temperature.

In [24]:
typesOfCrimes = processedSDF\
.filter((col("LOC_OF_OCCUR_DESC") != "INSIDE"))\
.select("OFNS_DESC").distinct().toPandas()

In [52]:
output = []

for crime in typesOfCrimes["OFNS_DESC"]:
    print(crime)
    crimeSpecific = outsideCrimes\
    .filter(col("OFNS_DESC") == crime)\
    .groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
    .count()\
    .sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
    .toPandas()
    
    if len(crimeSpecific) == 192:
        crimeSpecific["Date"] = crimeSpecific.CMPLNT_FR_MONTH.map(str) + "/" + crimeSpecific.CMPLNT_FR_YEAR.map(str)
        crimeSpecific.Date = pd.to_datetime(crimeSpecific.Date)
        crimeSpecific.set_index("Date", inplace=True)
        crimeSpecific.drop(columns=["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"], inplace=True)

        merged = crimeSpecific.merge(df.rename(columns={"DATE": "Date"}).set_index("Date"), left_index=True, right_index=True)
        ols = smf.ols(formula="TAVG ~ count", data=merged)
        model=ols.fit()
        output.append((crime, model.rsquared))

OFFENSES AGAINST PUBLIC ORDER/ADMINISTRATION


OTHER TRAFFIC INFRACTION
ANTICIPATORY OFFENSES


FELONY SEX CRIMES
NEW YORK CITY HEALTH CODE


VEHICLE AND TRAFFIC LAWS


KIDNAPPING & RELATED OFFENSES
HOMICIDE-NEGLIGENT-VEHICLE


...

HARRASSMENT


GRAND LARCENY


ADMINISTRATIVE CODES
KIDNAPPING
BURGLARY


ADMINISTRATIVE CODE


ABORTION
UNDER THE INFLUENCE OF DRUGS
KIDNAPPING AND RELATED OFFENSES


In [53]:
sorted(output, key=lambda x: x[1], reverse=True)

[('ASSAULT & RELATED OFFENSES', 0.8316418286144611),
 ('FELONY ASSAULT', 0.8200541738261642),
 ('HARRASSMENT', 0.734673859892066),
 ('MISCELLANEOUS PENAL LAW', 0.6009765365375949),
 ('SEX CRIMES', 0.5325505388308467),
 ('OFFENSES AGAINST THE PERSON', 0.5078619099821404),
 ('PETIT LARCENY', 0.48139985639595073),
 ('GRAND LARCENY', 0.2824157721636119),
 ('UNAUTHORIZED USE OF A VEHICLE', 0.27759742449478353),
 ('RAPE', 0.2751013712289899),
 ('CRIMINAL MISCHIEF', 0.2322374541273703),
 ('OFFENSES AGAINST PUBLIC ADMINISTRATION', 0.22264389152883024),
 ('OFFENSES AGAINST PUBLIC ORDER/ADMINISTRATION', 0.1899220929614457),
 ('POSSESSION OF STOLEN PROPERTY', 0.1867639682245552),
 ('BURGLARY', 0.17079596406676567),
 ('ROBBERY', 0.16128383689587),
 ('ADMINISTRATIVE CODE', 0.16082673224291222),
 ('VEHICLE AND TRAFFIC LAWS', 0.1508794678965628),
 ('FRAUDS', 0.1031032495217793),
 ('GRAND LARCENY OF MOTOR VEHICLE', 0.08401109637229254),
 ('DANGEROUS DRUGS', 0.06700278835177709),
 ('DANGEROUS WEAPONS',

In [42]:
assaultDF = outsideCrimes.filter(col("OFNS_DESC") == "ASSAULT & RELATED OFFENSES")\
.groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
.count()\
.sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
.toPandas()

assaultDF["Date"] = assaultDF.CMPLNT_FR_MONTH.map(str) + "/" + assaultDF.CMPLNT_FR_YEAR.map(str)
assaultDF.Date = pd.to_datetime(assaultDF.Date)
assaultDF.set_index("Date", inplace=True)
assaultDF.drop(columns=["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"], inplace=True)

In [43]:
fig = px.scatter(
    x=df.TAVG, 
    y=assaultDF["count"],
    trendline="ols",
    title="Crimes per Month vs. Temperature",
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Crime Rate",
)
fig

In [54]:
assaultDF = outsideCrimes.filter(col("OFNS_DESC") == "INTOXICATED & IMPAIRED DRIVING")\
.groupBy(["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"])\
.count()\
.sort([col("CMPLNT_FR_YEAR"), col("CMPLNT_FR_MONTH")])\
.toPandas()

assaultDF["Date"] = assaultDF.CMPLNT_FR_MONTH.map(str) + "/" + assaultDF.CMPLNT_FR_YEAR.map(str)
assaultDF.Date = pd.to_datetime(assaultDF.Date)
assaultDF.set_index("Date", inplace=True)
assaultDF.drop(columns=["CMPLNT_FR_YEAR", "CMPLNT_FR_MONTH"], inplace=True)

In [55]:
fig = px.scatter(
    x=df.TAVG, 
    y=assaultDF["count"],
    trendline="ols",
    title="Crimes per Month vs. Temperature",
)

fig.update_layout(
    xaxis_title="Temperature (°C)",
    yaxis_title="Crime Rate",
)
fig

In [ ]:
KIDNAPPING AND RELATED OFFENSES